# Ver 1 using char-cnn model

In [145]:
from theano.sandbox import cuda
%matplotlib inline
from __future__ import division, print_function
import operator

import numpy as np
from keras.layers import TimeDistributed, Activation
from numpy.random import choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from numpy.random import choice
import time

In [20]:
text = open('./data/lgqm-sample.txt').read()
text = unicode(text, "utf-8")

In [21]:
print('length of text: ', len(text), text[:100])

length of text:  35397 《临高启明》吹牛者

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何


In [33]:
charset = set(text)
char_freq_map = {c: 0 for c in charset}

In [34]:
print('size of charset', len(chars), chars[:40])

size of charset 2064 [[u'\u6000'], [u'\u6302'], [u'\u6d89'], [u'\u8c08'], [u'\u908b'], [u'\u968f'], [u'\u6297'], [u'\u6599'], [u'\u6d1e'], [u'\u9020'], [u'\u77a5'], [u'\u822c'], [u'\u4f5c'], [u'\u6cb3'], [u'\u6b32'], [u'\u4fb5'], [u'\u4e34'], [u'\u7136'], [u'\u5438'], [u'\u573a'], [u'\u5bbd'], [u'\u7ebf'], [u'\u53cd'], [u'\u76cf'], [u'\u79d1'], [u'\u82d7'], [u'\u82e5'], [u'\u8bdd'], [u'\u8d5e'], [u'\u51e1'], [u'\u74e3'], [u'\u77e5'], [u'\u9664'], [u'\u7f6a'], [u'\u626c'], [u'\u5176'], [u'\u95f9'], [u'\u7eff'], [u'\u6e14'], [u'\u6101']]


In [37]:
for c in text:
    char_freq_map[c] += 1

In [56]:
char_freq_list = sorted(char_freq_map.items(), key=operator.itemgetter(1))
char_freq_list.reverse()
charlist = [pair[0] for pair in char_freq_list]

In [57]:
char2idx = dict((c, i) for i, c in enumerate(charlist))

In [58]:
text_in_idx = [char2idx[c] for c in text]

[1041, 248, 20, 315, 40, 1222]

In [69]:
#text_in_idx[:6]
print(''.join(charlist[i] for i in text_in_idx[:24]))

《临高启明》吹牛者

严正声明：本书为丫丫小


In [89]:
maxlen = 48
sentences = []
next_chars = []
for i in range(0, len(text_in_idx) - maxlen):
    sentences.append(text_in_idx[i: i + maxlen])
    next_chars.append(text_in_idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 35349


In [90]:
sentences = np.array(sentences)
next_chars = np.array(next_chars)
sentences.shape, next_chars.shape

((35349, 48), (35349, 48))

In [135]:
n_fac = 100
batch_size = 64
n_hidden = 512
vocab_size = len(charlist)

In [136]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

In [137]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 48, 100)       206400      embedding_input_3[0][0]          
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 48, 512)       1255424     embedding_3[0][0]                
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 48, 512)       0           lstm_3[0][0]                     
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (None, 48, 512)       2099200     dropout_1[0][0]                  
___________________________________________________________________________________________

In [131]:
# max_sentence = len(sentences)//batch_size*batch_size

# max_sentence

# next_chars_adjusted = np.expand_dims(next_chars, -1)

# next_chars.shape, next_chars_adjusted.shape

((35349, 48), (35349, 48, 1))

In [138]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock
INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/lock_dir/lock


Epoch 1/1
35349/35349 [==============================] - 114s - loss: 5.3020   


In [158]:
def run_epochs(n):
    for i in range(n):
        model.reset_states()
        h = model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)
        print(h.history, h.history['loss'])
        weight_file_name = './data/text-generation-lgqm-ver-1-char-cnn-'+str(time.time())+'.h5'
        model.save_weights(weight_file_name)
        print('saved weights: ', weight_file_name)

In [147]:
run_epochs(1)

Epoch 1/1
35349/35349 [==============================] - 114s - loss: 2.9416   
{'loss': [2.9416295884100632]} [2.9416295884100632]


In [148]:
def print_example():
    seed_string = u"第一百二十八节 "
    for i in range(800):
        x=np.array([char2idx[c] for c in seed_string[-7:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [156]:
def print_example_inline_seed():
    pre_string = u"第65535节 "
    
    rand_idx = np.random.randint(0, high=len(text)-maxlen-1)
    seed_string = text[rand_idx:rand_idx+maxlen];
    for i in range(maxlen+1):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    seed_string = pre_string+seed_string[maxlen:]
    for i in range(800):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [149]:
model.optimizer.lr=0.05

In [150]:
run_epochs(1)

Epoch 1/1
35349/35349 [==============================] - 115s - loss: 2.5405   
{'loss': [2.5405140206980339]} [2.5405140206980339]


In [157]:
print_example_inline_seed()

第65535节 子山一一允诺。

    这些故说，旅起门成议？想来的心腹，但是比在明代最筑了。

    “也许说萧某行毕竟是是用活价气的－式各十件1的一个古代社会真能是二十两成的塑料、一下六工用，虫洞之惧，入海洞的帽子、酒店-澳洲海商社会潮替出宝，岂不算是因为安静，犹发一般，诸股行位在澳洲本的宵禁克拉克瓷，等于科镜的府墙工做在猎中的大益。这位保来的批货物的拍口的地方。萧子山却还是老观私要，文德嗣刚说到几个人腿肚门都有了一个澄论，高老爷看道：“几位精刻都恶愤碗。血肉模糊的作为在现历进，完比。

    查不要说现些出种的局面？”

    萧子山提来相向问题，不急穿越，能能随成了。便是考计了产下了一遍，一两老爷看说：“此如何面创了他五钱的渠在太据什么呢，没什么鬼选巧热子，论及才能人发入异实利静。实际上是每法贵之。再惜更他们欲绍成之礼。

    萧子山连王洛宾一脸神往，给台湾是一次手里，哪怕是这里看守更少钱。

    3、有丰空！其是他们身期监咬，他自己查当地。他们流动的冰方口理车的价格，现政府面后台湾做来代穿越的发言背。

    萧子山这真是江西捡在宽外。

    严作价海后，多家主义？可以拥市人都并不过大生品容在观。

    “是三个人腿，都有价下。另多克拉克瓷，还是还是因此刻里不定的。”

    濠、是是四十两吧。高老爷想下一眼建意的，只请暂识起出乎眼入的座穿越上。文德嗣决定首的一些闪口的产量的小票。平时换是要追意状是再马时期，一钱，不越豪度，那是挑非老这盘子没什么呢？只是前险发大之处决关贿赂大吏，仿山很多年，但是没有可能也但是崇祯，有样。不过半下煤油灯来贩船的什么出一步的批量制用、止区男、套小。虽然三尺，一件木忙斩主，在能把一件烟世界、各式各同分数35999是册人之战。[搜索最新更新尽在]

    这边已经是起察，对穿越了鄙论群。

    “灯拱点期往设作严重，不马两名艺才可以好广州。

    高老爷端详了一
